In [ ]:

import random
import sys
import string
from typing import List, Dict, Tuple, Set

player_name: str = "Player"            
player_score: int = 0                 

player_lives: int = 3                 
hints_available: int = 3               
skip_tokens: int = 2                  
streak: int = 0                        
asked_words: Set[str] = set()      

LEVELS: Tuple[str, ...] = ("Easy", "Medium", "Hard")

BASE_POINTS: Dict[str, int] = {"Easy": 10, "Medium": 15, "Hard": 20}
MULTIPLIER: Dict[str, int] = {"Easy": 1, "Medium": 2, "Hard": 3}


WORD_BANK: Dict[str, List[Dict[str, List[str]]]] = {
    "Easy": [
        {"english": "hello", "answers": ["bonjour"]},
        {"english": "goodbye", "answers": ["au revoir", "aurevoir", "au-revoir"]},
        {"english": "cat", "answers": ["chat"]},
        {"english": "dog", "answers": ["chien"]},
        {"english": "thank you", "answers": ["merci"]},
        {"english": "yes", "answers": ["oui"]},
        {"english": "no", "answers": ["non"]},
    ],
    "Medium": [
        {"english": "apple", "answers": ["pomme"]},
        {"english": "house", "answers": ["maison"]},
        {"english": "water", "answers": ["eau"]},
        {"english": "school", "answers": ["ecole", "école"]},
        {"english": "mother", "answers": ["mere", "mère"]},
        {"english": "father", "answers": ["pere", "père"]},
    ],
    "Hard": [
        {"english": "I love you", "answers": ["je t'aime", "je taime", "je t aime"]},
        {"english": "See you later", "answers": ["a plus tard", "à plus tard", "aplustard"]},
        {"english": "Happy birthday", "answers": ["joyeux anniversaire"]},
        {"english": "Excuse me / Sorry", "answers": ["pardon", "desole", "désolé", "desolé"]},
        {"english": "I am hungry", "answers": ["j'ai faim", "jai faim"]},
    ],
}

PUNCT_TO_REMOVE = str.maketrans("", "", string.punctuation)

def normalize(text: str) -> str:
    """
    Normalize text for comparison:
    - lowercase
    - remove punctuation
    - replace common French accents with plain letters
    - collapse multiple spaces
    """
    if not isinstance(text, str):
        return ""
    s = text.strip().lower()
    
    s = s.translate(PUNCT_TO_REMOVE)
    
    accents = {
        "à": "a", "â": "a", "ä": "a",
        "é": "e", "è": "e", "ê": "e", "ë": "e",
        "î": "i", "ï": "i",
        "ô": "o", "ö": "o",
        "ù": "u", "û": "u", "ü": "u",
        "ç": "c", "œ": "oe", "æ": "ae",
        "'": "", "’": ""
    }
    for a, r in accents.items():
        s = s.replace(a, r)
    
    s = " ".join(s.split())
    return s


def is_correct_answer(given: str, answers: List[str]) -> bool:
    """Compare normalized given answer to any normalized accepted answers."""
    ng = normalize(given)
    for a in answers:
        if ng == normalize(a):
            return True
    return False


def get_hint(answer: str) -> str:
    """Return a friendly hint for the answer: first letter(s) and length."""
    clean = normalize(answer)
    
    words = clean.split()
    if len(words) == 1:
        return f"Starts with '{clean[0]}' and is {len(clean)} letters long."
    else:
        # reveal first letters of each word
        firsts = " ".join(w[0] for w in words)
        lengths = " ".join(str(len(w)) for w in words)
        return f"Words start with: {firsts}. Word lengths: {lengths}."


def ask_player(prompt: str, valid_options: List[str] = None) -> str:
    
    while True:
        try:
            resp = input(prompt).strip()
        except (EOFError, KeyboardInterrupt):
            print("\nInput interrupted. Exiting game.")
            sys.exit(0)
        if resp == "":
            print("Please type something (or 'quit' to exit).")
            continue
        if resp.lower() == "quit":
            print("Quitting game. Bye!")
            sys.exit(0)
        if valid_options:
            if resp.lower() in valid_options:
                return resp.lower()
            else:
                print(f"Please enter one of: {', '.join(valid_options)}")
                continue
        return resp


def show_status():
    
    print("\n==== Status ====")
    print(f"Player: {player_name}  |  Score: {player_score}  |  Lives: {player_lives}")
    print(f"Hints: {hints_available}  |  Skips: {skip_tokens}  |  Streak: {streak}")
    print("================\n")

def ask_question(question: Dict[str, List[str]], level: str) -> bool:
    """
    Ask one translation question.
    Returns True if player answered correctly (affects score/streak outside).
    Accept commands:
      - 'hint' to use a hint
      - 'skip' to use a skip token
      - 'quit' to exit game (handled in ask_player)
    """
    global hints_available, skip_tokens, player_lives, player_score, streak

    english = question["english"]
    answers = question["answers"]
    base = BASE_POINTS[level]
    multiplier = MULTIPLIER[level]
    points = base * multiplier
    penalty = max(1, points // 2)  # penalty is half the question value (at least 1)

    print(f"\nTranslate to French: '{english}'")
    attempts = 0
    max_attempts = 2

    while attempts < max_attempts:
        attempts += 1
        prompt = f"Your answer (or type 'hint'/'skip'/'quit') [{attempts}/{max_attempts}]: "
        resp = ask_player(prompt)
        resp_lc = resp.lower()

        # handle hint
        if resp_lc == "hint":
            if hints_available <= 0:
                print("No hints left.")
                attempts -= 1  # don't count attempt if they only asked for hint and had none
                continue
            hints_available -= 1
            print("HINT:", get_hint(answers[0]))
            # allow another attempt without counting this as an additional attempt
            attempts -= 1
            continue

        # handle skip
        if resp_lc == "skip":
            if skip_tokens <= 0:
                print("No skip tokens left.")
                attempts -= 1
                continue
            skip_tokens -= 1
            print("Question skipped. No points awarded or lost.")
            # skipped => neither correct nor wrong, streak resets
            streak = 0
            return False

        # check answer
        if is_correct_answer(resp, answers):
            # correct: award points and possibly streak bonus
            player_score += points
            streak += 1
            print(f"Correct! +{points} points. (Total: {player_score})")
            # streak bonus: every 3 in a row => extra points
            if streak > 0 and streak % 3 == 0:
                bonus = 5 * multiplier
                player_score += bonus
                print(f"Streak bonus! You answered {streak} in a row: +{bonus} extra points.")
            return True
        else:
            # wrong
            player_score -= penalty
            player_lives -= 1  # lose a life for wrong answer
            streak = 0
            print(f"Not quite. -{penalty} points, and you lost a life. Lives left: {player_lives}. (Score: {player_score})")
            # if still have attempts, continue; otherwise return False
            if player_lives <= 0:
                print("You've run out of lives.")
                return False
            if attempts < max_attempts:
                print("Try again (or use 'hint'/'skip').")
            else:
                print("No more attempts for this question.")
    return False


def select_level() -> str:
    """Prompt the player to choose a level. Returns level string."""
    print("Choose a level:")
    for i, lv in enumerate(LEVELS, start=1):
        print(f"  {i}. {lv}")
    choice = ask_player("Enter level number (1-3): ", valid_options=[str(i) for i in range(1, len(LEVELS)+1)])
    idx = int(choice) - 1
    return LEVELS[idx]


def run_game():
    """Main game loop."""
    global player_name, player_score, player_lives, hints_available, skip_tokens, streak, asked_words

    print("Welcome to the English-French Translation Game! (type 'quit' anytime to exit)")
    name_inp = ask_player("What's your name? ")
    player_name = name_inp or player_name
    print(f"Bonjour, {player_name}! Let's learn some French. 🇫🇷\n")

    level = select_level()
    print(f"Level chosen: {level}. You will be asked up to 6 questions for this session.\n")

    pool = WORD_BANK.get(level, [])
    if not pool:
        print("No words available for this level. Exiting.")
        return
    random.shuffle(pool)
    max_questions = min(6, len(pool))
    questions = pool[:max_questions]

    for q in questions:
        
        if player_lives <= 0:
            print("\nNo more lives. Game over!")
            break

        if q["english"] in asked_words:
            continue
        asked_words.add(q["english"])
        ask_question(q, level)

    print("\n-- Session complete --")
    print(f"Player: {player_name}")
    print(f"Final Score: {player_score}")
    print(f"Lives left: {player_lives}")
    print(f"Hints remaining: {hints_available}, Skips remaining: {skip_tokens}")
    if player_score <= 0:
        print("Keep practicing — you'll get better!")
    else:
        print("Well done! Keep practicing your French words every day. 😊")

    print("\n(Programming demo: this game used variables, lists/dicts/tuples/sets, conditionals, loops, functions.)")

if __name__ == "__main__":
    run_game()

Welcome to the English-French Translation Game! (type 'quit' anytime to exit)
